# SAT model implementation details and notes

Implementatino with Z3 and CVC5 using the common pythonic API

Files:
- `solve.py`:
    - `solve_satisfy`: find the correct schedule
    - `solve_optimize`: find the optimal schedule
- `model.py`: main function with    
    1. argument parsing
        - `--solvers`: z3, MiniSat, CaDiCal
        - `--timeout`: timeout in seconds
        - `--teams`: number of teams
    2. round robin tournament `rb` definition
    3. `solve_satisfy`, `solve_optimize` calls for each solver specified in `--solvers`
    4. results dump to json file
- `sat_encodings.py`: propositional logic encodings of:
    - `at_most_one`: pairwise encoding, used only by `at_most_one_he` (Lecture 9, page 6)
    - `at_least_one`: $\bigvee_{i \in \N} x_i$
    - `at_most_one_he`: heule encoding (Lecture 9, page 12)
    - `at_most_k_seq`: sequential encoding (Lecture 9, page 21)
    - `exactly_one_he`: `at_most_one_he` $\wedge$ `at_least_one`
    - `Iff(A, B)`: macro for $(A \rightarrow B) \wedge (B \rightarrow A)$
    - `at_least_k_seq`: sequential encoding. \
    Let $\{x_i \ | i \in [1,N]\}$ be the variables on which we enforce $AtLeastK$, $\{s_{i,j} \ | \ i \in [1,N] \ j \in [1,K]\}$ the auxiliary variables for $K\geq2$.

        $$
            (s_{1,1} \leftrightarrow x_{1}) \ \wedge 
            \bigwedge_{i \in [2, N]}((s_{i-1, 1} \vee x_{i}) \leftrightarrow s_{i, 1}) \ \wedge 
            \bigwedge_{j \in [1, K]}(s_{N, j}) \ \wedge
            \bigwedge_{j \in [2, K]}(\neg s_{1, j}) \ \wedge
            \bigwedge_{j \in [2, k]}
                \bigwedge_{i \in [2, N]}[((s_{i-1, j-1} \wedge x_{i}) \vee s_{i-1, j})) \leftrightarrow s_{i, j}]
        $$

        $s_{i,j}=true \leftrightarrow$ the sequence $(x_1, ..., x_i)$ satisfies $AtLeastJ$.\
        Base cases:
        - $s_{1,1} \leftrightarrow x_1$ for the sequence $(x_{1})$.\
        - $\bigwedge_{j \in [1, K]}(s_{N, j})$, i.e. for $(x_1, ..., x_N)$, $AtLeastJ$ must hold $\forall j \in [1,K]$
        - $\bigwedge_{j \in [2, K]}(\neg s_{1, j})$, i.e. for any unit-length sequence $AtLeastJ$, with $j \geq 2$, can't hold


        For $i > 1, j > 1$ $s_{i, j} = true$ in one of the two following cases:
        - the sequence $(x_1, ..., x_{i-1})$ already satisfies $AtLeastJ$
        - the sequence $(x_1, ..., x_{i-1})$ satisfies $AtLeast(J-1)$ and $x_i=true$

        The encoding ensures that $\forall j \in [1, K]$ $AtLeastJ$ holds for a subsequence of $(x_1, ..., x_N)$
- `utils.py`: contains utility functions:
    - `gen_var_id`: used in `sat_encodings` to guarantee that all the auxiliary literals across the encodings of `sat_encodings` have unique identifiers. E.g the literals `s[i][j]` are found both in `at_most_k_seq` and `at_least_k_seq`, which may be used by the same model.
    - `set_params`: used to set solver specific parameters (unused)
    - `parse_matches_schedule`: given the model found in `solve_satisfy`, parse the schedule as a matrix
    - `parse_slots_schedule`: given the model found in `solve_optimize` and the matrix parsed from `solve_satisfy`, parse the optimal schedule as a matrix  

**NOTE**: Since the code in `utils.py` and `sat_encodings.py` is very verbose and not so important to the core process, I prefer not to report it for the sake of clarity.

## `main.py` routine

Argument parsing

In [ ]:
# Command line parameters
parser = argparse.ArgumentParser(description="STS with SAT")
parser.add_argument("-n", "--teams", type=int, help="Number of teams", required=True)
parser.add_argument(
    "-t",
    "--timeout",
    type=int,
    default=300,
    help="Solver timeout in seconds",
)
parser.add_argument(
    "--solvers",
    nargs="+",
    type=str,
    choices=["z3", "minisat", "cadical"],
    help="Solvers chosen to solve instance",
    required=True,
)

args = parser.parse_args()

n = args.teams  # number of teams
timeout = args.timeout  # Solver timeout
solvers = args.solvers  # list of solvers

teams = range(n)  # Team identifiers
weeks = range(n - 1)  # Week identifiers
periods = range(n // 2)  # Period identifiers

Round robin tournament. Slots are ordered for `solve_satisfy` and `solve_optimize` routines

In [ ]:
# Round robin tournament of sorted matches (t1,t2),
# where t1 < t2 and t1 plays at home, t2 plays away
rb = []
for p in periods:
    matches = []
    for w in weeks:
        if p == 0:
            matches.append((sorted([n - 1, w])))
        else:
            matches.append((sorted([(p + w) % (n - 1), (n - p + w - 1) % (n - 1)])))
    rb.append(matches)

For every solver in the list `solvers`:
1. call `solve_satisfy (rb, n, teams, weeks, periods, solver_mod_name, solver_params)`, where
    - `solver_mod_name`: either `"z3"` or `"cvc5.pythonic"`, i.e. CVC5 with MiniSat/CaDiCaL
    - `solver_params`: additional solver parameters (unused)

    It returns the solution `sat_schedule`, i.e. $sts$, and the resulting running time `sat_runtime`
2. If $sts$ is found, call `solve_optimize(sat_schedule,n,teams,weeks,periods,solution,solver_mod_name,solver_params)`. It returns the optimal schedule

In [ ]:
for solver_name in solvers:
    match solver_name:
        case "z3":
            solver_mod_name = "z3"
            solver_params = {}
        case _:  # i.e. minisat, cadical, implemented by cvc5
            solver_mod_name = "cvc5.pythonic"
            solver_params = {
                "sat_backend": [  # set backend sat solver of cvc5
                    "sat-solver",
                    solver_name,
                ],
            }
    # Reset total runtime for solver_name
    total_runtime = 0

    # Find basic schedule without scheduling the slots by calling solve_satisfy
    # using a child process.
    # If the solution isn't found within the time limit, save empty results and
    # try the next solver_name
    with Pool(1) as pool:
        result = pool.apply_async(
            solve_satisfy,
            (rb, n, teams, weeks, periods, solver_mod_name, solver_params),
        )
        try:
            # Wait the child process
            solution = result.get(timeout=timeout)
        except TimeoutError:
            print(f"{solver_name}: Timeout for satisfiability")
            solution = None

    if solution:
        sat_schedule, sat_runtime = solution[0], solution[1]
        print(f"{solver_name}: solve_satisfy in {sat_runtime:.4f}s")

        # Balance the number of times each team plays at home and away starting from
        # the satisfying solution of solve_satisfy using a child process.
        # If the solution isn't found within the time limit, try the next solver_name
        with Pool(1) as pool:
            result = pool.apply_async(
                solve_optimize,
                (
                    sat_schedule,
                    n,
                    teams,
                    weeks,
                    periods,
                    solution,
                    solver_mod_name,
                    solver_params,
                ),
            )
            try:
                # Wait the child process, considering the time remaining
                # from solve_satisfy
                solution = result.get(timeout=timeout - sat_runtime)
            except TimeoutError:
                print(f"{solver_name}: Timeout for optimization")
            else:
                opt_runtime = solution[1]
                print(f"{solver_name}: solve_optimize in {opt_runtime:.4f}s")
                total_runtime += opt_runtime

        total_runtime += sat_runtime

        print(f"{solver_name}: total runtime {total_runtime:.4f}s", end="\n\n")

    # Save results for current solver
    results[solver_name] = format_solution(solution, total_runtime)

Finally dump the results to N.json

In [ ]:
# Dump results to n.json, n being the number of teams
if results:
    with open(f"res/SAT/{n}.json", "w") as file:
        json.dump(results, file)

## `solve.py` routines

### `solve_satisfy`

This command is needed since `solve_satisfy` is run in a subprocess

In [ ]:
# translate module name into module
solver = importlib.import_module(solver_mod_name)

**Variables**:
- `matches_schedule[p][w][t]`, i.e. $match\_schedule_{p, w, t}$
- `matches_to_periods[ti][tj][p]`, i.e. $match\_to\_periods_{t_1, t_2, p}$

In [ ]:

# Variables
matches_schedule = [
    [[solver.Bool(f"matches_schedule_{p}_{w}_{t}") for t in periods] for w in weeks]
    for p in periods
]
matches_to_periods = [
    [
        [solver.Bool(f"matches_to_periods_{ti}_{tj}_{p}") for p in periods]
        for tj in teams
    ]
    for ti in teams
]


Instantiate solver with additional parameters (`solver_params` can be empty)

In [ ]:
 # solver for satisfiability
s_sat = solver.Solver()
set_params(s_sat, solver_params)

Every match is scheduled to a single period and each period is assigned to a single match

In [ ]:
# one-hot encoding of the index to select the match of the week (auxiliary constraint)
for p in periods:
    for w in weeks:
        s_sat.add(
            exactly_one_he([matches_schedule[p][w][t] for t in periods], solver)
        )
# for each week every match in rb[*][w] is scheduled once
for w in weeks:
    for t in periods:
        s_sat.add(
            exactly_one_he([matches_schedule[p][w][t] for p in periods], solver)
        )

Bind `matches_schedule` to `matches_to_periods`

In [ ]:
# bind matches_schedule with matches_to_periods (auxiliary constraint)
for p in periods:
    for w in weeks:
        for t in periods:
            s_sat.add(
                Iff(
                    matches_schedule[p][w][t],
                    matches_to_periods[rb[t][w][0]][rb[t][w][1]][p],
                    solver,
                )
            )

Every team plays at most twice in the same period: note that this implementation relies on the slot-wise ordering of `rb`

In [ ]:
# each team plays at most twice in the same period
for bp in periods:
    # constraint for the first team
    s_sat.add(
        at_most_k_seq(
            [matches_to_periods[0][t2][bp] for t2 in range(1, n)], 2, solver
        )
    )
    # constraint for the last team
    s_sat.add(
        at_most_k_seq(
            [matches_to_periods[t2][-1][bp] for t2 in range(0, n - 1)], 2, solver
        )
    )
    # constraint for the teams between the first and the last teams
    for t1 in range(1, n - 1):
        s_sat.add(
            at_most_k_seq(
                # matches where t1 is away, i.e. t1 > t2
                [matches_to_periods[t2][t1][bp] for t2 in range(t1)] +
                # matches where t1 is home, i.e. t1 < t2
                [matches_to_periods[t1][t2][bp] for t2 in range(t1 + 1, n)],
                2,
                solver,
            )
        )

If a solution is found, return the schedule with the runtime, otherwise return None

In [ ]:
start_time = time.time() # start solving
if s_sat.check() == solver.sat: # solving ends with the return value from s_sat.check()
    # Solution found
    runtime = time.time() - start_time
    model_sat = s_sat.model()
    sat_schedule = parse_matches_schedule( # parse the sat schedule
        matches_schedule, rb, periods, weeks, model_sat
    )
    return (
        sat_schedule,
        runtime,
        None, # this indicates that the solution is not balanced (i.e. optimized)
    )
else:
    # Solution not found
    return None

### `solve_optimize`

This command is needed since `solve_optimize` is run in a subprocess

In [ ]:
# translate module name into module
solver = importlib.import_module(solver_mod_name)

**Variables**
- `slots_schedule[p][w]` i.e. $flip\_slot_{p, w}$
- `match_to_slots[ti][tj]` i.e. $match\_to\_slots_{t1, t2}$

In [ ]:
 # Variables
slots_schedule = [
    [solver.Bool(f"slots_schedule_{p}_{w}") for w in weeks] for p in periods
]
matches_to_slots = [
    [solver.Bool(f"matches_to_slots_{ti}_{tj}") for tj in teams] for ti in teams
]

The optimal solution is set to `sat_solution` in the case where the optimization phase isn't completed

In [ ]:
 # opt_solution is sat_solution if optimization fails
opt_solution = sat_solution

Initialize the `runtime` to keep track of the elapsed time

In [ ]:
runtime = 0

Binary search of the maximum minimum number `mid` of times each team plays at home and away in the interval [`low`, `high`].\
The search pattern consists of:
- instantiating a new solver `s_opt` for each value of `mid`
- binding `slots_schedule` to `matches_to_slots` by considering the `sat_schedule` found by `solve_satisfy`
- imposing for each team the lower bound `mid` for:
    - the times playes away, i.e. `at_least_k_seq(matches_to_slots,mid,...)`
    - the times playes at home, i.e. `at_least_k_seq(Not(matches_to_slots),mid,...)`

As shown before, the implementation relies on the slot-wise ordering of `sat_schedule`

In [ ]:
 # Binary search optimization
low, high = 1, (n - 1) // 2
while low <= high:
    mid = (low + high) // 2

    s_opt = solver.Solver()  # solver for optimization
    set_params(s_opt, solver_params) 

    # bind slots_schedule to matches_to_slots
    for p in periods:
        for w in weeks:
            s_opt.add(
                Iff(
                    slots_schedule[p][w],
                    matches_to_slots[sat_schedule[p][w][0] - 1][
                        sat_schedule[p][w][1] - 1
                    ],
                    solver,
                )
            )

    # optimization: constrain the minimum number of times each team plays at home
    # first team
    s_opt.add(
        at_least_k_seq([matches_to_slots[0][t2] for t2 in range(1, n)], mid, solver) # lower bound on times played away
    )
    s_opt.add(
        at_least_k_seq([solver.Not(matches_to_slots[0][t2]) for t2 in range(1, n)], mid, solver) # lower bound on times played at home
    )
    # last team
    s_opt.add(
        at_least_k_seq(
            [solver.Not(matches_to_slots[t2][-1]) for t2 in range(0, n - 1)],
            mid,
            solver,
        )
    )
    s_opt.add(
        at_least_k_seq(
            [matches_to_slots[t2][-1] for t2 in range(0, n - 1)],
            mid,
            solver,
        )
    )
    # teams between the first and the last teams
    for t1 in range(1, n - 1):
        s_opt.add(
            # matches in sat_schedule where t1 is away, i.e. t1 > t2
            at_least_k_seq(
                [solver.Not(matches_to_slots[t2][t1]) for t2 in range(t1)]
                + [matches_to_slots[t1][t2] for t2 in range(t1 + 1, n)],
                mid,
                solver,
            )
        )
        s_opt.add(
            # matches in sat_schedule where t1 is home, i.e. t1 < t2
            at_least_k_seq(
                [matches_to_slots[t2][t1] for t2 in range(t1)]
                + [solver.Not(matches_to_slots[t1][t2]) for t2 in range(t1 + 1, n)],
                mid,
                solver,
            )
        )
    if s_opt.check() == solver.sat:
        runtime += time.time() - start_time # update run-time
        model_opt = s_opt.model()
        sat_schedule = sat_solution[0]
        opt_schedule = parse_slots_schedule(
            sat_schedule, slots_schedule, periods, weeks, model_opt
        )
        opt_solution = (  # update optimal solution
            opt_schedule,
            runtime,
            mid,
        )
        low = mid + 1  # increase optimization function
    else:
        runtime += time.time() - start_time
        high = mid - 1  # roll back

return opt_solution

# Conclusion

- Symmetry breaking constraints aren't implemented since no consistent computational gain was observed.